In [1]:
from sklearn import preprocessing
from sklearn import linear_model,svm,gaussian_process
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor,BaggingRegressor,GradientBoostingRegressor,ExtraTreesRegressor
from sklearn.model_selection import KFold,cross_val_score,train_test_split,StratifiedKFold
from sklearn.linear_model import ElasticNet, Lasso, Ridge
from sklearn import metrics
from sklearn.svm import SVR, LinearSVR
from sklearn.linear_model import ElasticNet, SGDRegressor, BayesianRidge
from sklearn.kernel_ridge import KernelRidge
from xgboost import XGBRegressor#XGB迴歸也來一組參數
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.pipeline import Pipeline, make_pipeline
from scipy.stats import skew
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso, LassoCV
from sklearn.preprocessing import RobustScaler,StandardScaler
%matplotlib inline
from sklearn.feature_selection import SelectFromModel

In [2]:
import pandas as pd
from sklearn import datasets
data = pd.read_csv('C:\\Users\\kikok\\Downloads\\dataset\\train.csv')

In [3]:

cols=['building_area','land_area','building_complete_dt'
      ,'txn_dt','village_income_median','lon','total_floor','lat',
      'txn_floor','XIV_10000'
      ,'town','XIV_5000','parking_price','XIII_10000',
      'town_area','town_population_density','XIII_5000','XIV_1000','village','XIII_MIN']
cols2=['building_area','land_area','parking_price','village_income_median']
cols3=['parking_area','parking_price','txn_floor']
'''   
cols_rfr=['building_area','XIII_10000','building_complete_dt'
      ,'land_area','jobschool_rate','XIII_5000','txn_dt','village_income_median',
      'V_10000','elementary_rate'
      ,'txn_floor','junior_rate','divorce_rate','highschool_rate',
      'VII_1000','V_5000','master_rate','XIV_5000','marriage_rate','bachelor_rate']
cols=['building_area','highschool_rate','elementary_rate'
      ,'master_rate','bachelor_rate','lon','village_income_median','XIII_5000',
      'jobschool_rate','X_10000'
      ,'II_10000','XII_10000','III_1000','XIII_10000',
      'IX_10000','VII_10000','VIII_10000','XI_10000','V_10000','VI_10000']
'''
for col in cols3:
    data[col].fillna(0, inplace=True)
data['village_income_median'].fillna(data['village_income_median'].median(), inplace=True)    
'''
data["oTotal_floor"] = data.total_floor.map({2:1,8:1,18:1, 
                                         16:2, 3:2, 4:2,5:2,9:2,10:2,
                                         6:3,12:3 ,13:3, 19:3,20:3,21:3, 
                                         1:4,7:4 ,14:4, 15:4,17:4,24:4,29:4,  
                                         11:5,23:5 ,22:5, 25:5,  
                                         26:6,27:6 ,28:6})
data["oTxn_floor"] = data.txn_floor.map({2:1,3:1,4:1,5:1, 
                                         26:2, 16:2, 14:2,0:2,6:2,8:2,
                                         1:3,7:3 ,9:3, 10:3,11:3,12:3, 
                                         13:4,21:4 ,22:4, 15:4,17:4,  
                                         18:5,19:5 ,20:5, 24:5,  
                                         23:6,25:6,27:6 ,28:6})
data["oCity"] = data.city.map({7:1,9:1, 
                               3:2, 5:2, 13:2,17:2,
                               6:3,10:3 ,14:3, 21:3,
                               12:4})
data =data.drop(['txn_floor','city','total_floor'],axis=1)

data[cols2] = np.log1p(data[cols2])
'''

'\ndata["oTotal_floor"] = data.total_floor.map({2:1,8:1,18:1, \n                                         16:2, 3:2, 4:2,5:2,9:2,10:2,\n                                         6:3,12:3 ,13:3, 19:3,20:3,21:3, \n                                         1:4,7:4 ,14:4, 15:4,17:4,24:4,29:4,  \n                                         11:5,23:5 ,22:5, 25:5,  \n                                         26:6,27:6 ,28:6})\ndata["oTxn_floor"] = data.txn_floor.map({2:1,3:1,4:1,5:1, \n                                         26:2, 16:2, 14:2,0:2,6:2,8:2,\n                                         1:3,7:3 ,9:3, 10:3,11:3,12:3, \n                                         13:4,21:4 ,22:4, 15:4,17:4,  \n                                         18:5,19:5 ,20:5, 24:5,  \n                                         23:6,25:6,27:6 ,28:6})\ndata["oCity"] = data.city.map({7:1,9:1, \n                               3:2, 5:2, 13:2,17:2,\n                               6:3,10:3 ,14:3, 21:3,\n                         

In [4]:
scaler = RobustScaler()
data2 =data.drop(['building_id','total_price'],axis=1)
x = data2
y = np.log1p(data['total_price'])
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.33,random_state = 42)
X_train_scaled = scaler.fit(x).transform(X_train)
X_test_scaled = scaler.fit(x).transform(X_test)  

In [5]:
clf = Pipeline([
  ('feature_selection', SelectFromModel(XGBRegressor(n_estimators=500, max_features=0.3))),
  ('predict', XGBRegressor(max_depth=20)),
])
clf.fit(X_train_scaled,y_train)

Pipeline(memory=None,
     steps=[('feature_selection', SelectFromModel(estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, max_features=0.3, min_child_weight=1, missing=None,
       n_estimators=500, n_jobs=...
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1))])

In [6]:
#clf1=XGBRegressor(n_estimators=10000,max_depth=20,min_child_weight=2,gamma=0.1)
clf.fit(X_train_scaled,y_train)
#print("{}: {:.6f}".format("xgb",xgb.score(X_test_scaled,y_test)))
print("{}: {:.6f}".format("xgb",clf.score(X_test_scaled,y_test)))

xgb: 0.962840


In [7]:
test_data = pd.read_csv('C:\\Users\\kikok\\Downloads\\dataset\\test.csv')
for col in cols3:
    test_data[col].fillna(0, inplace=True)
test_data['village_income_median'].fillna(test_data['village_income_median'].median(), inplace=True)    
'''
test_data["oTotal_floor"] = test_data.total_floor.map({2:1,8:1,18:1, 
                                         16:2, 3:2, 4:2,5:2,9:2,10:2,
                                         6:3,12:3 ,13:3, 19:3,20:3,21:3, 
                                         1:4,7:4 ,14:4, 15:4,17:4,24:4,29:4,  
                                         11:5,23:5 ,22:5, 25:5,  
                                         26:6,27:6 ,28:6})
test_data["oTxn_floor"] = test_data.txn_floor.map({2:1,3:1,4:1,5:1, 
                                         26:2, 16:2, 14:2,0:2,6:2,8:2,
                                         1:3,7:3 ,9:3, 10:3,11:3,12:3, 
                                         13:4,21:4 ,22:4, 15:4,17:4,  
                                         18:5,19:5 ,20:5, 24:5,  
                                         23:6,25:6,27:6 ,28:6})
test_data["oCity"] = test_data.city.map({7:1,9:1, 
                               3:2, 5:2, 13:2,17:2,
                               6:3,10:3 ,14:3, 21:3,
                               12:4})
test_data2 =test_data.drop(['txn_floor','city','total_floor','building_id'],axis=1)
'''
test_data2 = test_data.drop(['building_id'],axis=1)
test_data2[cols2] = np.log1p(test_data2[cols2])
test_x_predict = test_data2.values
test_x_predict_scaled = scaler.fit(test_x_predict).transform(test_x_predict)  

In [8]:
#y_ridge = rigit.predict(x_test_data)
#y_rf = np.expm1(clf.predict(test_x_predict_scaled))
#y_bagging = bag.predict(x_test_data)
#y_adaboost = ada.predict(x_test_data)
y_xgb =  np.expm1(clf.predict(test_x_predict_scaled))
#y_Extra =  np.expm1(Extra.predict(test_x_predict_scaled))
#y_stack = np.expm1(stack_model.predict(test_x_predict_scaled))

In [10]:
y_xgb

array([ 5805021.5  ,  4035395.75 ,  7430225.   , ...,  1281889.625,
        2359612.5  ,  2794602.25 ], dtype=float32)

In [12]:
prediction = pd.DataFrame(y_xgb,columns=['total_price'])
result = pd.concat((test_data['building_id'],prediction),axis=1)


In [13]:

result.to_csv('C:\\Users\\kikok\\Downloads\\dataset\\submit_test_20190514__0.962840_xgb.csv',index=False)

In [ ]:
class stacking(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self,mod,meta_model):
        self.mod = mod
        self.meta_model = meta_model
        self.kf = KFold(n_splits=5, random_state=42, shuffle=True)
        
    def fit(self,X,y):
        self.saved_model = [list() for i in self.mod]
        oof_train = np.zeros((X.shape[0], len(self.mod)))
        
        for i,model in enumerate(self.mod):
            for train_index, val_index in self.kf.split(X,y):
                renew_model = clone(model)
                renew_model.fit(X[train_index], y[train_index])
                self.saved_model[i].append(renew_model)
                oof_train[val_index,i] = renew_model.predict(X[val_index])
        
        self.meta_model.fit(oof_train,y)
        return self
    
    def predict(self,X):
        whole_test = np.column_stack([np.column_stack(model.predict(X) for model in single_model).mean(axis=1) 
                                      for single_model in self.saved_model]) 
        return self.meta_model.predict(whole_test)
    
    def get_oof(self,X,y,test_X):
        oof = np.zeros((X.shape[0],len(self.mod)))
        test_single = np.zeros((test_X.shape[0],5))
        test_mean = np.zeros((test_X.shape[0],len(self.mod)))
        for i,model in enumerate(self.mod):
            for j, (train_index,val_index) in enumerate(self.kf.split(X,y)):
                clone_model = clone(model)
                clone_model.fit(X[train_index],y[train_index])
                oof[val_index,i] = clone_model.predict(X[val_index])
                test_single[:,j] = clone_model.predict(test_X)
            test_mean[:,i] = test_single.mean(axis=1)
        return oof, test_mean

In [ ]:
from sklearn.preprocessing import Imputer
a = Imputer().fit_transform(X_train_scaled)
b = Imputer().fit_transform(y_train.values.reshape(-1,1)).ravel()

In [ ]:
def rmse_cv(model,X,y):
    rmse = np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=5))
    return rmse

lasso = Lasso(alpha=0.0005,max_iter=10000)
ridge = Ridge(alpha=60)
svr = SVR(gamma= 0.0004,kernel='rbf',C=13,epsilon=0.009)
ker = KernelRidge(alpha=0.2 ,kernel='polynomial',degree=3 , coef0=0.8)
ela = ElasticNet(alpha=0.005,l1_ratio=0.08,max_iter=10000)
bay = BayesianRidge()
xgb =  XGBRegressor(max_depth=20)
rfr = RandomForestRegressor(n_estimators=500, max_features=0.3)

stack_model = stacking(mod=[rfr,ela,lasso,ridge],meta_model=xgb)
score = rmse_cv(stack_model,a,b)
print(score.mean())

In [ ]:
stack_model.fit(a,b)
print("{}: {:.6f}".format("stack_model",stack_model.score(X_test_scaled,y_test)))

In [ ]:
def rmse_cv(model,X,y):
    rmse = np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=5))
    return rmse
models = [Ridge(),Lasso(alpha=0.01,max_iter=10000),RandomForestRegressor(),GradientBoostingRegressor(),SVR(),LinearSVR(),
          ElasticNet(alpha=0.001,max_iter=10000),SGDRegressor(max_iter=1000,tol=1e-3),BayesianRidge(),KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5),
          ExtraTreesRegressor(),XGBRegressor()]

names = [ "Ridge", "Lasso", "RF", "GBR", "SVR", "LinSVR", "Ela","SGD","Bay","Ker","Extra","Xgb"]
for name, model in zip(names, models):
    score = rmse_cv(model, X_train_scaled, y_train)
    print("{}: {:.6f}, {:.4f}".format(name,score.mean(),score.std()))

In [ ]:
#y_final = (y_rf + y_xgb) / 2

In [ ]:
max_features = [.1, .3, .5, .7, .9, .99]
test_scores = []
for max_feat in max_features:
    clf = RandomForestRegressor(n_estimators=200, max_features=max_feat)
    test_score = np.sqrt(-cross_val_score(clf, X_train_scaled, y_train, cv=5, scoring='neg_mean_squared_error'))
    test_scores.append(np.mean(test_score))
plt.plot(max_features, test_scores)
plt.title("Max Features vs CV Error");

In [ ]:
alphas_alt = [-0.01,0.001,0.005,0.01,0.02,0.05,1,1.5]
kfolds = KFold(n_splits=10, shuffle=True, random_state=42)
test_scores = []
for alpha in alphas_alt:
    clf = Ridge(alpha)
    test_score = np.sqrt(-cross_val_score(clf, X_train_scaled, y_train, cv=kfolds, scoring='neg_mean_squared_error'))
    test_scores.append(np.mean(test_score))
plt.plot(alphas_alt, test_scores)
plt.title("alphas_alt vs CV Error");

In [ ]:
rigit = Ridge(1)
params = [1,2,3,4,5,6,10,20]
test_scores = []
for param in params:
    clf = XGBRegressor(max_depth=param,base_estimator=rigit)
    test_score = np.sqrt(-cross_val_score(clf, X_train_scaled, y_train, cv=kfolds,scoring='neg_mean_squared_error'))
    test_scores.append(np.mean(test_score))
plt.plot(params, test_scores)
plt.title("param vs CV Error");

In [ ]:
rigit = Ridge(1)
xgb =XGBRegressor(max_depth=20,base_estimator=rigit)
xgb.fit( X_train_scaled, y_train)
rfr = RandomForestRegressor(n_estimators=500, max_features=0.3)
rfr.fit( X_train_scaled, y_train)

In [ ]:
class skew_dummies(BaseEstimator, TransformerMixin):
    def __init__(self,skew=0.5):
        self.skew = skew
    
    def fit(self,X,y=None):
        return self
    
    def transform(self,X):
        X_numeric=X.select_dtypes(exclude=["object"])
        skewness = X_numeric.apply(lambda x: skew(x))
        skewness_features = skewness[abs(skewness) >= self.skew].index
        X[skewness_features] = np.log1p(X[skewness_features])
        X = pd.get_dummies(X)
        return X
pipe = Pipeline([('skew_dummies', skew_dummies(skew=1)),])    
   

In [ ]:

data2 =data.drop(['lat'],axis=1)
data_pipe = pipe.fit_transform(data2)